In [1]:
import pandas as pd
import json

In [18]:
vertica_df = pd.read_excel("output.xlsx")
vertica_df.head()

,tfdUUID,cfdi
0,0010F845-637E-4A10-A7C4-258C34F4F0A9,"{\n ""_TipoRelacion"": ""01"",\n ""Cf..."
1,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,"{\n \n ""_TipoRelacion"": ""04"",\n ..."
2,012BCE87-BE31-46B1-88E0-9AE7C65E5213,"{\n ""_TipoRelacion"": ""07"",\n ""Cf..."
3,012D915A-7397-4E08-8A78-10E42B5D659F,"{\n ""_TipoRelacion"": ""01"",\n ""Cf..."
4,0008E8E6-3681-4833-9EF1-3F9C973E85B5,"{\n ""_TipoRelacion"": ""01"",\n ""Cf..."


In [19]:

def find_dicts_with_keys(d, keys):
    found_dicts = []
    if isinstance(d, dict):
        if any(key in d for key in keys):
            found_dicts.append(d)
        for value in d.values():
            if isinstance(value, (dict, list)):
                found_dicts.extend(find_dicts_with_keys(value, keys))
    elif isinstance(d, list):
        for item in d:
            found_dicts.extend(find_dicts_with_keys(item, keys))
    return found_dicts



def extract_keys_from_dicts(dicts, keys_to_extract):
    extracted_dicts = []
    for d in dicts:
        extracted_dict = {}
        for key in keys_to_extract:
            if key in d:
                extracted_dict[key] = d[key]
        extracted_dicts.append(extracted_dict)
    return extracted_dicts


In [20]:
tfduuid = vertica_df['tfdUUID'][1]
cfdi_Text = vertica_df['cfdi'][1]
cfdi_Json = json.loads(cfdi_Text)
cfdi_Json
keys=['_TipoRelacion']
f = find_dicts_with_keys(cfdi_Json, keys)
ext = extract_keys_from_dicts(f, keys)
ext

[{'_TipoRelacion': '04'}]

In [21]:
cfdi_Text = vertica_df['cfdi'][2]
keys=["_UUID"]
cfdi_Json = json.loads(cfdi_Text)
f_id = find_dicts_with_keys(cfdi_Json, keys)
f_id
    

[{'_UUID': '8DBEC254-20DA-49BD-8731-40509F74CDF8'},
 {'_UUID': '94AB5055-F5CA-4588-B6EB-D826F91BED00'},
 {'_UUID': 'CBA7778F-8969-47BB-A683-58D59F1C6A02'},
 {'_UUID': '191597A8-3D30-4CA2-93A1-894CC1354C67'},
 {'_UUID': 'C4B007AB-156E-488F-A457-C841E6A6C6E9'},
 {'_UUID': 'EE70D9B9-CC0E-4604-A4EA-A16B8F395B7C'},
 {'_UUID': 'CD086CFB-ED58-4B97-9C78-DEDC418D2AC5'}]

In [22]:
cfdi_Text = vertica_df['cfdi'][3]

cfdi_Json = json.loads(cfdi_Text)
new = []
if "0" in cfdi_Json:
    for i in list(cfdi_Json.keys()):
        new_dict = cfdi_Json[i]
        k_rel=['_TipoRelacion']
        f_rel = find_dicts_with_keys(new_dict, k_rel)
        ext_rel = extract_keys_from_dicts(f_rel, k_rel)
        for dic in ext_rel:
            k_uuid=["_UUID"]
            f_id = find_dicts_with_keys(new_dict, k_uuid)
            new.append({**dic, "uuids":f_id})
else:
    k_rel=['_TipoRelacion']
    f_rel = find_dicts_with_keys(cfdi_Json, k_rel)
    ext_rel = extract_keys_from_dicts(f_rel, k_rel)
    for dic in ext_rel:
        k_uuid=["_UUID"]
        f_id = find_dicts_with_keys(cfdi_Json, k_uuid)
        new.append({**dic, "uuids":f_id})
new

[{'_TipoRelacion': '01',
  'uuids': [{'_UUID': '3AFDE537-2A8C-4E15-B038-542439BF695A'}]}]

## Función que agrega diccionarios con listas

In [23]:
def add_dicts_on_list(cfdi_Json): 
    new = []
    if "0" in cfdi_Json:
        for i in list(cfdi_Json.keys()):
            new_dict = cfdi_Json[i]
            new.append(add_dicts_on_list(new_dict))
        new = [dic for sub_l in new for dic in sub_l ]
    else:
        k_rel=['_TipoRelacion']
        f_rel = find_dicts_with_keys(cfdi_Json, k_rel)
        ext_rel = extract_keys_from_dicts(f_rel, k_rel)
        for dic in ext_rel:
            k_uuid=["_UUID"]
            f_id = find_dicts_with_keys(cfdi_Json, k_uuid)
            new.append({**dic, "uuids":f_id})
    return new

In [24]:
cfdi_Text = vertica_df['cfdi'][2]
cfdi_Json = json.loads(cfdi_Text)
add_dicts_on_list(cfdi_Json)

[{'_TipoRelacion': '07',
  'uuids': [{'_UUID': '8DBEC254-20DA-49BD-8731-40509F74CDF8'},
   {'_UUID': '94AB5055-F5CA-4588-B6EB-D826F91BED00'},
   {'_UUID': 'CBA7778F-8969-47BB-A683-58D59F1C6A02'},
   {'_UUID': '191597A8-3D30-4CA2-93A1-894CC1354C67'},
   {'_UUID': 'C4B007AB-156E-488F-A457-C841E6A6C6E9'},
   {'_UUID': 'EE70D9B9-CC0E-4604-A4EA-A16B8F395B7C'},
   {'_UUID': 'CD086CFB-ED58-4B97-9C78-DEDC418D2AC5'}]}]

In [25]:
cfdi_Text = vertica_df['cfdi'][3]
cfdi_Json = json.loads(cfdi_Text)
add_dicts_on_list(cfdi_Json)

[{'_TipoRelacion': '01',
  'uuids': [{'_UUID': '3AFDE537-2A8C-4E15-B038-542439BF695A'}]}]

## Función que crea diccionarios para convertirlos a data frame

In [26]:
nu = 3
tfduuid = vertica_df['tfdUUID'][nu]
cfdi_Text = vertica_df['cfdi'][nu]
cfdi_Json = json.loads(cfdi_Text)
to_procces = add_dicts_on_list(cfdi_Json)


def create_dictionaries_to_df(to_procces,tfduuid):
    uuid = []
    id_relacionados = []
    tipoRelacion = []
    _uids = []
    try:
        if len(to_procces) == 1:
            dic = to_procces[0]
            relacion = dic['_TipoRelacion']
            for j,dic_uid in enumerate(dic["uuids"]):
                uuid.append(tfduuid)
                id_relacionados.append(j+1)
                tipoRelacion.append(relacion)
                #key_uid_numeric =f"{j+1}"
                key_uid_numeric = 1   
                _uids.append([{key_uid_numeric:dic_uid['_UUID']}])
            new_extracted_dic = {"UUID":uuid, "id_relacionados":id_relacionados, "tipoRelacion":tipoRelacion, "_uid":_uids}

        else:
            for j,dicts in enumerate(to_procces):
                relacion = dicts['_TipoRelacion']
                uuid.append(tfduuid)
                id_relacionados.append(j+1)
                tipoRelacion.append(relacion)
                uuid_dicts = dicts['uuids']
                sub_uids = []
                for k,dic_uid in enumerate(uuid_dicts):
                    #key_uid_numeric =f"{k+1}"
                    key_uid_numeric =k+1
                    sub_uids.append({key_uid_numeric:dic_uid['_UUID']})
                _uids.append(sub_uids)
                new_extracted_dic = {"UUID":uuid, "id_relacionados":id_relacionados, "tipoRelacion":tipoRelacion, "_uid":_uids}
                
    except Exception as e:
        new_extracted_dic={"UUID":[tfduuid], "id_relacionados":[0],"tipoRelacion":["0"], "_uid":[{1:'--Nan--'}] }
    return new_extracted_dic    

pd.DataFrame(create_dictionaries_to_df(to_procces,tfduuid))



,UUID,id_relacionados,tipoRelacion,_uid
0,012D915A-7397-4E08-8A78-10E42B5D659F,1,01,[{1: '3AFDE537-2A8C-4E15-B038-542439BF695A'}]


## Función uno trabaja sobre las filas

In [27]:
def functionOne_CfdiRelacionados(row):
    tfduuid = row['tfdUUID']
    try:
        cfdi_Text = row['cfdi']
        cfdi_Json = json.loads(cfdi_Text)
        add_dicts_to_procces_list = add_dicts_on_list(cfdi_Json)
        return create_dictionaries_to_df(add_dicts_to_procces_list,tfduuid)
    except Exception as e:
        print("Check functionOne") 
   


In [28]:
vertica_df

,tfdUUID,cfdi
0,0010F845-637E-4A10-A7C4-258C34F4F0A9,"{\n ""_TipoRelacion"": ""01"",\n ""Cf..."
1,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,"{\n \n ""_TipoRelacion"": ""04"",\n ..."
2,012BCE87-BE31-46B1-88E0-9AE7C65E5213,"{\n ""_TipoRelacion"": ""07"",\n ""Cf..."
3,012D915A-7397-4E08-8A78-10E42B5D659F,"{\n ""_TipoRelacion"": ""01"",\n ""Cf..."
4,0008E8E6-3681-4833-9EF1-3F9C973E85B5,"{\n ""_TipoRelacion"": ""01"",\n ""Cf..."
5,007B160A-C071-48FD-B0AF-D800AA4A1155,"{\n ""_TipoRelacion"": ""07"",\n ""Cf..."
6,00D3BEF8-18A2-4493-A280-AFBC4FE1EBF1,"\t{\n ""0"": {\n ""_TipoRelacio..."
7,01F6E34C-8343-4D96-86FB-9D1921231B64,"{\n\t""_TipoRelacion"": ""04"",\n\t""CfdiRelacionad..."
8,0453B575-81CF-489F-AE74-1AF1AA67374E,"{\n\t""_TipoRelacion"": ""04"",\n\t""CfdiRelacionad..."
9,054F7E17-44B3-442F-90DB-90A2C4DB8638,"{\n\t""_TipoRelacion"": ""04"",\n\t""CfdiRelacionad..."


## Función 2 que se aplica sobre todo el dataframe y devuelve una lista

In [29]:

def functionTwo_CfdiRelacionados(df):
    lists_dicts = df.apply(functionOne_CfdiRelacionados, axis=1).tolist()
    return lists_dicts
preview = functionTwo_CfdiRelacionados(vertica_df)
preview

[{'UUID': ['0010F845-637E-4A10-A7C4-258C34F4F0A9'],
  'id_relacionados': [1],
  'tipoRelacion': ['01'],
  '_uid': [[{1: 'E2655F4D-05CB-4F1C-A244-AD2778D4205E'}]]},
 {'UUID': ['010FF698-E3C6-4FAD-9AB5-0BDE5F93424A',
   '010FF698-E3C6-4FAD-9AB5-0BDE5F93424A'],
  'id_relacionados': [1, 2],
  'tipoRelacion': ['04', '04'],
  '_uid': [[{1: 'FB6420D5-B950-4498-A72D-B337FEEF6077'}],
   [{1: '6E8E10F5-F79D-4F0F-8E12-1B3383F90424'}]]},
 {'UUID': ['012BCE87-BE31-46B1-88E0-9AE7C65E5213',
   '012BCE87-BE31-46B1-88E0-9AE7C65E5213',
   '012BCE87-BE31-46B1-88E0-9AE7C65E5213',
   '012BCE87-BE31-46B1-88E0-9AE7C65E5213',
   '012BCE87-BE31-46B1-88E0-9AE7C65E5213',
   '012BCE87-BE31-46B1-88E0-9AE7C65E5213',
   '012BCE87-BE31-46B1-88E0-9AE7C65E5213'],
  'id_relacionados': [1, 2, 3, 4, 5, 6, 7],
  'tipoRelacion': ['07', '07', '07', '07', '07', '07', '07'],
  '_uid': [[{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'}],
   [{1: '94AB5055-F5CA-4588-B6EB-D826F91BED00'}],
   [{1: 'CBA7778F-8969-47BB-A683-58D59F1C6A02'

## Aplicación del explode para devolver el dataframe en formato deseado

### Hasta aquí se llenaría la primera tabla

In [30]:
df = pd.DataFrame(preview)
df = df.explode(['_uid','UUID', 'id_relacionados','tipoRelacion'], ignore_index=True)
df

,UUID,id_relacionados,tipoRelacion,_uid
0,0010F845-637E-4A10-A7C4-258C34F4F0A9,1,01,[{1: 'E2655F4D-05CB-4F1C-A244-AD2778D4205E'}]
1,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,1,04,[{1: 'FB6420D5-B950-4498-A72D-B337FEEF6077'}]
2,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,2,04,[{1: '6E8E10F5-F79D-4F0F-8E12-1B3383F90424'}]
3,012BCE87-BE31-46B1-88E0-9AE7C65E5213,1,07,[{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'}]
4,012BCE87-BE31-46B1-88E0-9AE7C65E5213,2,07,[{1: '94AB5055-F5CA-4588-B6EB-D826F91BED00'}]
5,012BCE87-BE31-46B1-88E0-9AE7C65E5213,3,07,[{1: 'CBA7778F-8969-47BB-A683-58D59F1C6A02'}]
6,012BCE87-BE31-46B1-88E0-9AE7C65E5213,4,07,[{1: '191597A8-3D30-4CA2-93A1-894CC1354C67'}]
7,012BCE87-BE31-46B1-88E0-9AE7C65E5213,5,07,[{1: 'C4B007AB-156E-488F-A457-C841E6A6C6E9'}]
8,012BCE87-BE31-46B1-88E0-9AE7C65E5213,6,07,[{1: 'EE70D9B9-CC0E-4604-A4EA-A16B8F395B7C'}]
9,012BCE87-BE31-46B1-88E0-9AE7C65E5213,7,07,[{1: 'CD086CFB-ED58-4B97-9C78-DEDC418D2AC5'}]


In [31]:
cfdirelacionados_dict ={
   'uuid': "",
   'idRelacionados': '',
   'tipoRelacion': ''
}
old_names = ['UUID', 'id_relacionados', 'tipoRelacion']
new_names =['uuid','idRelacionados','tipoRelacion']
num_cols = []
str_cols =  ["uuid",'tipoRelacion']
int_cols = ['idRelacionados']
ordered_cols = ['uuid', 'idRelacionados', 'tipoRelacion' ]
def re_build_df(df,merge_dict,old_names,new_names, num_cols, str_cols, int_cols, ordered_cols):

    for j, old_name in enumerate(old_names):
        new_name = new_names[j]
        df = df.rename(columns={old_name: new_name})

    merge_df = pd.DataFrame([merge_dict])
    df = pd.merge(merge_df,df, how="outer")
    df = df.drop(df.index[0])

    df  = df.fillna(0)
    for colu in df.columns:
        if colu in num_cols:
            df[colu] = df[colu].astype(float)
        elif colu in str_cols:
            df[colu] = df[colu].astype(str)
        elif colu in int_cols:
             df[colu] = df[colu].astype(int)    
            
    new_df = df[ordered_cols]
    return new_df
new_df = re_build_df(df=df,merge_dict = cfdirelacionados_dict, old_names = old_names, new_names=new_names, 
            num_cols=num_cols, str_cols = str_cols , int_cols = int_cols, ordered_cols  = ordered_cols )
new_df

,uuid,idRelacionados,tipoRelacion
1,0010F845-637E-4A10-A7C4-258C34F4F0A9,1,01
2,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,1,04
3,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,2,04
4,012BCE87-BE31-46B1-88E0-9AE7C65E5213,1,07
5,012BCE87-BE31-46B1-88E0-9AE7C65E5213,2,07
6,012BCE87-BE31-46B1-88E0-9AE7C65E5213,3,07
7,012BCE87-BE31-46B1-88E0-9AE7C65E5213,4,07
8,012BCE87-BE31-46B1-88E0-9AE7C65E5213,5,07
9,012BCE87-BE31-46B1-88E0-9AE7C65E5213,6,07
10,012BCE87-BE31-46B1-88E0-9AE7C65E5213,7,07


In [16]:
new_df.dtypes

uuid              object
idRelacionados     int32
tipoRelacion      object
dtype: object

## Aquí comienza el proceso para el llenado de la segunda tabla

In [32]:
uidRelacionado_df = df.explode(['_uid'], ignore_index=True)

uidRelacionado_df 

,UUID,id_relacionados,tipoRelacion,_uid
0,0010F845-637E-4A10-A7C4-258C34F4F0A9,1,01,{1: 'E2655F4D-05CB-4F1C-A244-AD2778D4205E'}
1,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,1,04,{1: 'FB6420D5-B950-4498-A72D-B337FEEF6077'}
2,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,2,04,{1: '6E8E10F5-F79D-4F0F-8E12-1B3383F90424'}
3,012BCE87-BE31-46B1-88E0-9AE7C65E5213,1,07,{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'}
4,012BCE87-BE31-46B1-88E0-9AE7C65E5213,2,07,{1: '94AB5055-F5CA-4588-B6EB-D826F91BED00'}
5,012BCE87-BE31-46B1-88E0-9AE7C65E5213,3,07,{1: 'CBA7778F-8969-47BB-A683-58D59F1C6A02'}
6,012BCE87-BE31-46B1-88E0-9AE7C65E5213,4,07,{1: '191597A8-3D30-4CA2-93A1-894CC1354C67'}
7,012BCE87-BE31-46B1-88E0-9AE7C65E5213,5,07,{1: 'C4B007AB-156E-488F-A457-C841E6A6C6E9'}
8,012BCE87-BE31-46B1-88E0-9AE7C65E5213,6,07,{1: 'EE70D9B9-CC0E-4604-A4EA-A16B8F395B7C'}
9,012BCE87-BE31-46B1-88E0-9AE7C65E5213,7,07,{1: 'CD086CFB-ED58-4B97-9C78-DEDC418D2AC5'}


In [33]:

uidRelacionado_df['uid'] = uidRelacionado_df['_uid'].apply(lambda x: list(x.values())[0])
uidRelacionado_df

,UUID,id_relacionados,tipoRelacion,_uid,uid
0,0010F845-637E-4A10-A7C4-258C34F4F0A9,1,01,{1: 'E2655F4D-05CB-4F1C-A244-AD2778D4205E'},E2655F4D-05CB-4F1C-A244-AD2778D4205E
1,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,1,04,{1: 'FB6420D5-B950-4498-A72D-B337FEEF6077'},FB6420D5-B950-4498-A72D-B337FEEF6077
2,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,2,04,{1: '6E8E10F5-F79D-4F0F-8E12-1B3383F90424'},6E8E10F5-F79D-4F0F-8E12-1B3383F90424
3,012BCE87-BE31-46B1-88E0-9AE7C65E5213,1,07,{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'},8DBEC254-20DA-49BD-8731-40509F74CDF8
4,012BCE87-BE31-46B1-88E0-9AE7C65E5213,2,07,{1: '94AB5055-F5CA-4588-B6EB-D826F91BED00'},94AB5055-F5CA-4588-B6EB-D826F91BED00
5,012BCE87-BE31-46B1-88E0-9AE7C65E5213,3,07,{1: 'CBA7778F-8969-47BB-A683-58D59F1C6A02'},CBA7778F-8969-47BB-A683-58D59F1C6A02
6,012BCE87-BE31-46B1-88E0-9AE7C65E5213,4,07,{1: '191597A8-3D30-4CA2-93A1-894CC1354C67'},191597A8-3D30-4CA2-93A1-894CC1354C67
7,012BCE87-BE31-46B1-88E0-9AE7C65E5213,5,07,{1: 'C4B007AB-156E-488F-A457-C841E6A6C6E9'},C4B007AB-156E-488F-A457-C841E6A6C6E9
8,012BCE87-BE31-46B1-88E0-9AE7C65E5213,6,07,{1: 'EE70D9B9-CC0E-4604-A4EA-A16B8F395B7C'},EE70D9B9-CC0E-4604-A4EA-A16B8F395B7C
9,012BCE87-BE31-46B1-88E0-9AE7C65E5213,7,07,{1: 'CD086CFB-ED58-4B97-9C78-DEDC418D2AC5'},CD086CFB-ED58-4B97-9C78-DEDC418D2AC5


In [36]:
# Crear una nueva columna "id" con las claves de los diccionarios
uidRelacionado_df ['id'] = uidRelacionado_df ['_uid'].apply(lambda x: list(x.keys())[0])
uidRelacionado_df 

,UUID,id_relacionados,tipoRelacion,_uid,uid,id
0,0010F845-637E-4A10-A7C4-258C34F4F0A9,1,01,{1: 'E2655F4D-05CB-4F1C-A244-AD2778D4205E'},E2655F4D-05CB-4F1C-A244-AD2778D4205E,1
1,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,1,04,{1: 'FB6420D5-B950-4498-A72D-B337FEEF6077'},FB6420D5-B950-4498-A72D-B337FEEF6077,1
2,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,2,04,{1: '6E8E10F5-F79D-4F0F-8E12-1B3383F90424'},6E8E10F5-F79D-4F0F-8E12-1B3383F90424,1
3,012BCE87-BE31-46B1-88E0-9AE7C65E5213,1,07,{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'},8DBEC254-20DA-49BD-8731-40509F74CDF8,1
4,012BCE87-BE31-46B1-88E0-9AE7C65E5213,2,07,{1: '94AB5055-F5CA-4588-B6EB-D826F91BED00'},94AB5055-F5CA-4588-B6EB-D826F91BED00,1
5,012BCE87-BE31-46B1-88E0-9AE7C65E5213,3,07,{1: 'CBA7778F-8969-47BB-A683-58D59F1C6A02'},CBA7778F-8969-47BB-A683-58D59F1C6A02,1
6,012BCE87-BE31-46B1-88E0-9AE7C65E5213,4,07,{1: '191597A8-3D30-4CA2-93A1-894CC1354C67'},191597A8-3D30-4CA2-93A1-894CC1354C67,1
7,012BCE87-BE31-46B1-88E0-9AE7C65E5213,5,07,{1: 'C4B007AB-156E-488F-A457-C841E6A6C6E9'},C4B007AB-156E-488F-A457-C841E6A6C6E9,1
8,012BCE87-BE31-46B1-88E0-9AE7C65E5213,6,07,{1: 'EE70D9B9-CC0E-4604-A4EA-A16B8F395B7C'},EE70D9B9-CC0E-4604-A4EA-A16B8F395B7C,1
9,012BCE87-BE31-46B1-88E0-9AE7C65E5213,7,07,{1: 'CD086CFB-ED58-4B97-9C78-DEDC418D2AC5'},CD086CFB-ED58-4B97-9C78-DEDC418D2AC5,1


In [37]:
uidRelacionado_df.columns

Index(['UUID', 'id_relacionados', 'tipoRelacion', '_uid', 'uid', 'id'], dtype='object')

In [38]:
uuidRelacionado_dict ={
   'uuid': "string",
   'idRelacionados': 0,
   'id': 0,
   '__uuid': "string"
}
old_names = ['UUID', 'id_relacionados','id',	'uid'	]
new_names =['uuid', 'idRelacionados', 'id', '_uuid']
num_cols = []
str_cols =  ["uuid",'__uid']
int_cols = ['id','idRelacionados']
ordered_cols = ['uuid', 'idRelacionados', 'id', '_uuid' ]
new_relacionado_df = re_build_df(df=uidRelacionado_df ,merge_dict = cfdirelacionados_dict, old_names = old_names, new_names=new_names, 
            num_cols=num_cols, str_cols = str_cols , int_cols = int_cols, ordered_cols  = ordered_cols )
new_relacionado_df

,uuid,idRelacionados,id,_uuid
1,0010F845-637E-4A10-A7C4-258C34F4F0A9,1,1,E2655F4D-05CB-4F1C-A244-AD2778D4205E
2,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,1,1,FB6420D5-B950-4498-A72D-B337FEEF6077
3,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,2,1,6E8E10F5-F79D-4F0F-8E12-1B3383F90424
4,012BCE87-BE31-46B1-88E0-9AE7C65E5213,1,1,8DBEC254-20DA-49BD-8731-40509F74CDF8
5,012BCE87-BE31-46B1-88E0-9AE7C65E5213,2,1,94AB5055-F5CA-4588-B6EB-D826F91BED00
6,012BCE87-BE31-46B1-88E0-9AE7C65E5213,3,1,CBA7778F-8969-47BB-A683-58D59F1C6A02
7,012BCE87-BE31-46B1-88E0-9AE7C65E5213,4,1,191597A8-3D30-4CA2-93A1-894CC1354C67
8,012BCE87-BE31-46B1-88E0-9AE7C65E5213,5,1,C4B007AB-156E-488F-A457-C841E6A6C6E9
9,012BCE87-BE31-46B1-88E0-9AE7C65E5213,6,1,EE70D9B9-CC0E-4604-A4EA-A16B8F395B7C
10,012BCE87-BE31-46B1-88E0-9AE7C65E5213,7,1,CD086CFB-ED58-4B97-9C78-DEDC418D2AC5


In [340]:
new_relacionado_df.dtypes

uuid              object
idRelacionados     int32
id                 int32
_uuid             object
dtype: object

## Apartir de aquí se debe subir a un a vertica

In [39]:
df = pd.DataFrame(preview)
df = df.explode(['UUID', 'id_relacionados', 'tipoRelacion'], ignore_index=True)
df

,UUID,id_relacionados,tipoRelacion,_uid
0,0010F845-637E-4A10-A7C4-258C34F4F0A9,1,01,[[{1: 'E2655F4D-05CB-4F1C-A244-AD2778D4205E'}]]
1,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,1,04,[[{1: 'FB6420D5-B950-4498-A72D-B337FEEF6077'}]...
2,010FF698-E3C6-4FAD-9AB5-0BDE5F93424A,2,04,[[{1: 'FB6420D5-B950-4498-A72D-B337FEEF6077'}]...
3,012BCE87-BE31-46B1-88E0-9AE7C65E5213,1,07,[[{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'}]...
4,012BCE87-BE31-46B1-88E0-9AE7C65E5213,2,07,[[{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'}]...
5,012BCE87-BE31-46B1-88E0-9AE7C65E5213,3,07,[[{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'}]...
6,012BCE87-BE31-46B1-88E0-9AE7C65E5213,4,07,[[{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'}]...
7,012BCE87-BE31-46B1-88E0-9AE7C65E5213,5,07,[[{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'}]...
8,012BCE87-BE31-46B1-88E0-9AE7C65E5213,6,07,[[{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'}]...
9,012BCE87-BE31-46B1-88E0-9AE7C65E5213,7,07,[[{1: '8DBEC254-20DA-49BD-8731-40509F74CDF8'}]...


In [ ]:
[{"_TipoRelacion": "01","lis":[{"_UUID": "E2655F4D-05CB-4F1C-A244-AD2778D4205E"}]}]
[{"_TipoRelacion": "04","lis":[{"_UUID": "FB6420D5-B950-4498-A72D-B337FEEF6077"},
                             {"_UUID": "6E8E10F5-F79D-4F0F-8E12-1B3383F90424"}
                             ]}]
[{"_TipoRelacion": "07","lis":[{"_UUID": "8DBEC254-20DA-49BD-8731-40509F74CDF8"},
                             {"_UUID": "94AB5055-F5CA-4588-B6EB-D826F91BED00"},
                             {"_UUID": "CBA7778F-8969-47BB-A683-58D59F1C6A02"},
                             {"_UUID": "191597A8-3D30-4CA2-93A1-894CC1354C67"},
                             {"_UUID": "C4B007AB-156E-488F-A457-C841E6A6C6E9"},
                             {"_UUID": "EE70D9B9-CC0E-4604-A4EA-A16B8F395B7C"},
                             {"_UUID": "CD086CFB-ED58-4B97-9C78-DEDC418D2AC5"} 
                            ]}]
[{"_TipoRelacion": "07","lis":[{"_UUID": "66A74B46-99F4-49F3-BDF5-68FBD65D24EE"}]},
 {"_TipoRelacion": "07","lis":[{"_UUID": "EE8A9F0F-E8F8-40D7-BEF4-67159CC65307"},
                            {"_UUID": "XX8A9F0F-E8F8-40D7-BEF4-67159CC65307"}
                            ]},
 {"_TipoRelacion": "07","lis": [{"_UUID": "3A10A93F-0562-47CD-9574-F5D85093FE21"}]},
 {"_TipoRelacion": "07","lis": [{"_UUID": "9C7090EB-BA17-40C1-AEE6-86EB5959EE82"}]},
 {"_TipoRelacion": "01","lis": [{"_UUID": "EE70D9B9-CC0E-4604-A4EA-A16B8F395B7C"},
                               {"_UUID": "CD086CFB-ED58-4B97-9C78-DEDC418D2AC5"}
                               ]}
        
]
